In [2]:
import os

from IDRnD.utils import Train, seed_everything
from IDRnD.augmentations import ToMellSpec, PadOrClip, ToTensor, Normalize_predef
from IDRnD.dataset import Test_Dataset, get_train_data
from IDRnD.resnet import resnet18

import numpy as np
import pandas as pd
import librosa
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

seed_everything(0)

In [4]:
post_transform = transforms.Compose([
    ToMellSpec(n_mels=128),
    Normalize_predef(-29.6179, 16.6342),
    librosa.power_to_db,
    PadOrClip(320),
    ToTensor(),
])

In [3]:
X, y = get_train_data()

In [ ]:
test_dataset = Test_Dataset(X)

In [7]:
hm = Train()
batch_size = 400
#test_dataset = Test_Dataset(eval_protocol["path"], post_transform)
test_dataset = Test_Dataset(X[:520], post_transform)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = resnet18(num_classes=1).cuda()

#model.load_state_dict(torch.load('models/simple_old_conv.pt'))
#model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
#torch.save(model_dst.module.state_dict(),  'models/kaggle2_nonparallel.pt') 
model.eval()
model.load_state_dict(torch.load('models/resnet_18_last_epoch.pt'))
pred = hm.predict_on_test(test_loader, model)